<table align="left">
  <tr>
    <td><img src="fleet.png" alt="fleet of icecream trucks" width="120"/></td>
    <td align="left"><h1>Lesson 4: Monitoring and Evaluating your Agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Setup Tracing

In [1]:
PROJECT_NAME = "Customer-Success"

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

load_dotenv() # load variables from local .env file


tracer_provider = register(
    project_name=PROJECT_NAME,
    # endpoint= get_phoenix_endpoint() + "v1/traces"
    endpoint = os.getenv('DLAI_LOCAL_URL').format(port='6006') + "/v1/traces"
)

# from phoenix.trace.tracer import register_tracer

# tracer_provider = register_tracer(
#     project_name="my-smolagent",
#     endpoint="http://localhost:6006/v1/traces"
# )

SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Phoenix Project: Customer-Success
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [4]:
from dotenv import load_dotenv, find_dotenv
load_dotenv() # load variables from local .env file

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

**Note: Must run "pheonix serve" first in command**

In [12]:
from smolagents import HfApiModel

model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

model([{"role": "user", "content": "Hello!"}])

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hello! How can I assist you today?', tool_calls=[], raw=ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Hello! How can I assist you today?', tool_call_id=None, tool_calls=[]), logprobs=None, seed=2305747842381961500)], created=1749073641, id='nwpebnN-4yUbBN-94aaa0d25867073b', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=ChatCompletionOutputUsage(completion_tokens=10, prompt_tokens=31, total_tokens=41), object='chat.completion', prompt=[]))

In [6]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

# print(os.getenv('DLAI_LOCAL_URL').format(port='6006'))

# url = os.get("DLAI_LOCAL_URL").format(port="6006")
# print(url)

http://localhost:6006


## Trace an agent run

In [7]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(model=model, tools=[])

>Note, the following line will sometimes get a timeout on the interface to the tracing package due to the networked interface. If this happens, try it again.


In [8]:
agent.run("What is the 100th Fibonacci number?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 100th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_100 = fibonacci(100)                                                                                         
  final_answer(fib_100)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 354224848179261915075

[Step 1: Duration 4.63 seconds| Input tokens: 2,020 | Output tokens: 130]

354224848179261915075

In [9]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

http://localhost:6006


## Setup ice cream production system

In [13]:
from smolagents import tool
from typing import Dict

menu_prices = {"crepe nutella": 1.50, "vanilla ice cream": 2, "maple pancake": 1.}

ORDER_BOOK = {}

@tool
def place_order(quantities: Dict[str, int], session_id: int) -> None:
    """Places a pre-order of snacks.

    Args:
        quantities: a dictionary with names as keys and quantities as values
        session_id: the id for the client session
    """
    global ORDER_BOOK
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    ORDER_BOOK[session_id] = quantities

@tool
def get_prices(quantities: Dict[str, int]) -> str:
    """Gets price for certain quantities of ice cream.

    Args:
        quantities: a dictionary with names as keys and quantities as values
    """
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    total_price = sum([menu_prices[key] * value for key, value in quantities.items()])
    return (
        f"Given the current menu prices:\n{menu_prices}\nThe total price for your order would be: ${total_price}"
    )

In [14]:
order_agent = CodeAgent(
    tools=[place_order, get_prices],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
)

In [15]:
order_agent.run(
    "Could I come and collect one crepe nutella?",
    additional_args={"session_id": 192}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 192}.                                                                                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  place_order(quantities={'crepe nutella': 1}, session_id=session_id)                                              
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 3.36 seconds| Input tokens: 2,193 | Output tokens: 68]

'Order placed for one crepe nutella.'

### Try multiple orders

In [16]:
client_requests = [
    ("Could I come and collect one crepe nutella?", "place_order"),
    ("What would be the price for 1 crêpe nutella + 2 pancakes?", "get_prices"),
    ("How did you start your ice-cream business?", None),
    ("What's the weather at the Louvre right now?", None),
    ("I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want it. If it's below, I'll order it, please.", "place_order")
]

In [18]:
for request in client_requests:
    order_agent.run(
        request[0],
        additional_args={"session_id": 0, "menu_prices": menu_prices}
    )

In [19]:
import phoenix as px

spans = px.Client().get_spans_dataframe(project_name=PROJECT_NAME)
spans.head(20)

name span_kind         parent_id  \
context.span_id                                                     
fd57bd4988fee51e  HfApiModel.__call__       LLM  cc5de8a1090a917c   
979d2d2858fe0cf5      FinalAnswerTool      TOOL  cc5de8a1090a917c   
cc5de8a1090a917c               Step 1     CHAIN  749fee021ff039b8   
749fee021ff039b8        CodeAgent.run     AGENT              None   
83b8daf2ca263b45  HfApiModel.__call__       LLM              None   
b892323134a01965  HfApiModel.__call__       LLM              None   
7cc748739990c3cc  HfApiModel.__call__       LLM  40461406f8ca0f9f   
800fb483c807801d           SimpleTool      TOOL  40461406f8ca0f9f   
f743aa80c0eba368      FinalAnswerTool      TOOL  40461406f8ca0f9f   
40461406f8ca0f9f               Step 1     CHAIN  2f1b6e5ae3c17800   
2f1b6e5ae3c17800        CodeAgent.run     AGENT              None   
d13753b69758e08e  HfApiModel.__call__       LLM  344d18187b222dea   
344d18187b222dea               Step 1     CHAIN  d369c6a5740f2a78   
d369c6a5740f2a78        CodeAgent.run     AGENT              None   

                                       start_time  \
context.span_id                                     
fd57bd4988fee51e 2025-06-04 21:44:46.318582+00:00   
979d2d2858fe0cf5 2025-06-04 21:44:50.925941+00:00   
cc5de8a1090a917c 2025-06-04 21:44:46.317975+00:00   
749fee021ff039b8 2025-06-04 21:44:46.252564+00:00   
83b8daf2ca263b45 2025-06-04 21:45:42.488546+00:00   
b892323134a01965 2025-06-04 21:47:20.814790+00:00   
7cc748739990c3cc 2025-06-04 21:48:48.168116+00:00   
800fb483c807801d 2025-06-04 21:48:51.502319+00:00   
f743aa80c0eba368 2025-06-04 21:48:51.508394+00:00   
40461406f8ca0f9f 2025-06-04 21:48:48.167807+00:00   
2f1b6e5ae3c17800 2025-06-04 21:48:48.144863+00:00   
d13753b69758e08e 2025-06-04 21:49:41.286115+00:00   
344d18187b222dea 2025-06-04 21:49:41.285695+00:00   
d369c6a5740f2a78 2025-06-04 21:49:41.269336+00:00   

                                         end_time status_code  \
context.span_id                                                 
fd57bd4988fee51e 2025-06-04 21:44:48.849427+00:00          OK   
979d2d2858fe0cf5 2025-06-04 21:44:50.926073+00:00          OK   
cc5de8a1090a917c 2025-06-04 21:44:50.932471+00:00          OK   
749fee021ff039b8 2025-06-04 21:44:50.941871+00:00          OK   
83b8daf2ca263b45 2025-06-04 21:45:42.612501+00:00       ERROR   
b892323134a01965 2025-06-04 21:47:21.523218+00:00          OK   
7cc748739990c3cc 2025-06-04 21:48:49.464051+00:00          OK   
800fb483c807801d 2025-06-04 21:48:51.502417+00:00          OK   
f743aa80c0eba368 2025-06-04 21:48:51.508482+00:00          OK   
40461406f8ca0f9f 2025-06-04 21:48:51.517019+00:00          OK   
2f1b6e5ae3c17800 2025-06-04 21:48:51.526626+00:00          OK   
d13753b69758e08e 2025-06-04 21:49:41.383497+00:00       ERROR   
344d18187b222dea 2025-06-04 21:49:43.442001+00:00       ERROR   
d369c6a5740f2a78 2025-06-04 21:49:43.457869+00:00       ERROR   

                                                     status_message  \
context.span_id                                                       
fd57bd4988fee51e                                                      
979d2d2858fe0cf5                                                      
cc5de8a1090a917c                                                      
749fee021ff039b8                                                      
83b8daf2ca263b45  HfHubHTTPError: 402 Client Error: Payment Requ...   
b892323134a01965                                                      
7cc748739990c3cc                                                      
800fb483c807801d                                                      
f743aa80c0eba368                                                      
40461406f8ca0f9f                                                      
2f1b6e5ae3c17800                                                      
d13753b69758e08e  HfHubHTTPError: 402 Client Error: Payment Requ...   
344d18187b222dea  AgentGenerationError: Erro

### Add processing to extract desired information

In [20]:
import pandas as pd
import json

agents = spans[spans['span_kind'] == 'AGENT'].copy()
agents['task'] = agents['attributes.input.value'].apply(
    lambda x: json.loads(x).get('task') if isinstance(x, str) else None
)

tools = spans.loc[
    spans['span_kind'] == 'TOOL',
    ["attributes.tool.name", "attributes.input.value", "context.trace_id"]
].copy()

tools_per_task = agents[
    ["name", "start_time", "task", "context.trace_id"]
].merge(
    tools,
    on="context.trace_id",
    how="left",
)
tools_per_task.head()

name                       start_time  \
0  CodeAgent.run 2025-06-04 21:44:46.252564+00:00   
1  CodeAgent.run 2025-06-04 21:48:48.144863+00:00   
2  CodeAgent.run 2025-06-04 21:48:48.144863+00:00   
3  CodeAgent.run 2025-06-04 21:49:41.269336+00:00   

                                          task  \
0          What is the 100th Fibonacci number?   
1  Could I come and collect one crepe nutella?   
2  Could I come and collect one crepe nutella?   
3  Could I come and collect one crepe nutella?   

                   context.trace_id attributes.tool.name  \
0  4ad48628b27d7f6106ed888f392948c4         final_answer   
1  52f210ea58e4bec38d3efdae9b1aaeeb          place_order   
2  52f210ea58e4bec38d3efdae9b1aaeeb         final_answer   
3  bdfd9ef868547d65cc5b585e05828ac4                  NaN   

                              attributes.input.value  
0  {"args": [354224848179261915075], "sanitize_in...  
1  {"args": [], "sanitize_inputs_outputs": false,...  
2  {"args": ["Order placed for one crepe nutella....  
3                                                NaN

### Now, compare tool calls with exected tool calls

In [21]:
def score_request(expected_tool: str, tool_calls: list):
    if expected_tool is None:
        return tool_calls == set(["final_answer"])
    else:
        return expected_tool in tool_calls

results = []
for request, expected_tool in client_requests:
    tool_calls = set(tools_per_task.loc[tools_per_task["task"] == request, "attributes.tool.name"].tolist())
    results.append(
        {
            "request": request,
            "tool_calls_performed": tool_calls,
            "is_correct": score_request(expected_tool, tool_calls)
        }
    )
pd.DataFrame(results)

request  \
0        Could I come and collect one crepe nutella?   
1  What would be the price for 1 crêpe nutella + ...   
2         How did you start your ice-cream business?   
3        What's the weather at the Louvre right now?   
4  I'm not sure if I should order. I want a vanil...   

               tool_calls_performed  is_correct  
0  {place_order, nan, final_answer}        True  
1                                {}       False  
2                                {}       False  
3                                {}       False  
4                                {}       False